In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')
import funciones as f

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
df_datos = pd.read_csv('../Datos/BD_SUICIDIOS_2007-2023 (1).csv', sep =',') # cargo el df

In [3]:
df_datos.info() # veo el contenido y el formato

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10035 entries, 0 to 10034
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CIE10      10035 non-null  object
 1   REGION     10035 non-null  int64 
 2   PROVINCIA  10035 non-null  int64 
 3   MUNICIPIO  10035 non-null  int64 
 4   DIASEMANA  10035 non-null  int64 
 5   MESOCU     10035 non-null  int64 
 6   ANOOCU     10035 non-null  int64 
 7   RHORA      10035 non-null  int64 
 8   SEXO       10035 non-null  int64 
 9   LUGAROCU   10035 non-null  int64 
 10  MEDIOEMP   10035 non-null  int64 
 11  MOTIVO     10035 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 940.9+ KB


In [4]:
df_datos.nunique()

CIE10         1
REGION       11
PROVINCIA    33
MUNICIPIO     8
DIASEMANA    12
MESOCU       17
ANOOCU       13
RHORA         2
SEXO         18
LUGAROCU      9
MEDIOEMP      9
MOTIVO       14
dtype: int64

- Hay algo que resalta, a partir de la columna municipio y es la cantidad de elementos diferentes dentro de cada una de las posteriores. Tomamos como ejemplo el SEXO, donde muestra que tiene 18 opciones diferentes.
- Eliminaré la primera columna porque solo tiene un valor

In [5]:
columnas = ['CIE10', 'REGION', 'PROVINCIA', 'MUNICIPIO', 'DIASEMANA', 'MESOCU', 'ANOOCU', 'RHORA', 'SEXO', 'LUGAROCU', 'MEDIOEMP', 'MOTIVO']
df = df_datos[columnas] #nuevo df

# insertar una nueva columna al final con valores NaN
df['GRUPOEDAD'] = None

# hacer el desplazamiento de columnas (de 'MUNICIPIO' a 'MOTIVO') una posición a la derecha
columnas_a_mover = columnas[3:]  # desde 'MUNICIPIO' en adelante

for i in range(len(columnas_a_mover) - 1, -1, -1):
    origen = columnas_a_mover[i - 1]  # columna anterior
    destino = columnas_a_mover[i]
    df[destino] = df[origen]

# vaciar la columna 'MUNICIPIO' original
df['MUNICIPIO'] = None

In [6]:
df.nunique()

CIE10         1
REGION       11
PROVINCIA    33
MUNICIPIO     0
DIASEMANA     8
MESOCU       12
ANOOCU       17
RHORA        13
SEXO          2
LUGAROCU     18
MEDIOEMP      9
MOTIVO        9
GRUPOEDAD     0
dtype: int64

In [7]:
df['GRUPOEDAD'] = df_datos['MOTIVO']
df.nunique()

CIE10         1
REGION       11
PROVINCIA    33
MUNICIPIO     0
DIASEMANA     8
MESOCU       12
ANOOCU       17
RHORA        13
SEXO          2
LUGAROCU     18
MEDIOEMP      9
MOTIVO        9
GRUPOEDAD    14
dtype: int64

In [8]:
df.drop(columns=['CIE10', 'MUNICIPIO'], inplace=True)

In [9]:
df.columns = df.columns.str.lower()

## Traer lista de df del diccionario

In [10]:
# nombres de archivos
filenames = [
    'df_diasem.csv',
    'df_mes.csv',
    'df_rhora.csv',
    'df_region.csv',
    'df_provincia.csv',
    'df_lugar.csv',
    'df_medio.csv',
    'df_genero.csv',
    'df_grupoedad.csv',
    'df_motivo.csv'
]

# crear un diccionario para almacenar los DataFrames leídos
df_dict = {}

# leer cada archivo y almacenarlo en el diccionario
for filename in filenames:
    df_name = filename.split('.')[0]  # usar el nombre del archivo (sin extensión) como clave
    df_dict[df_name] = pd.read_csv(f'../Datos/{filename}', index_col=0)

In [11]:
df_lleno = df.copy()

In [12]:
# lista de tuplas con las columnas a mapear, el df correspondiente y la columna única
mapeos = [
    ('motivo', df_dict['df_motivo']), ('region', df_dict['df_region']), ('provincia', df_dict['df_provincia']), ('diasemana', df_dict['df_diasem']), ('mesocu', df_dict['df_mes']), ('rhora', df_dict['df_rhora']),
    ('lugarocu', df_dict['df_lugar']), ('sexo', df_dict['df_genero']),('grupoedad', df_dict['df_grupoedad']),('medioemp', df_dict['df_medio']),
]

for columna, df_mapeado in mapeos:
    print(f"Procesando columna: {columna}")

    mapeo = dict(zip(df_mapeado['Valor'], df_mapeado['Significado']))

    valores = df_lleno[columna].unique()
    mapeo_filtrado = {k: v for k, v in mapeo.items() if k in valores}

    df_lleno[columna] = df_lleno[columna].map(mapeo_filtrado)

Procesando columna: motivo
Procesando columna: region
Procesando columna: provincia
Procesando columna: diasemana
Procesando columna: mesocu
Procesando columna: rhora
Procesando columna: lugarocu
Procesando columna: sexo
Procesando columna: grupoedad
Procesando columna: medioemp


In [13]:
df_lleno

,region,provincia,diasemana,mesocu,anoocu,rhora,sexo,lugarocu,medioemp,motivo,grupoedad
0,Región Cibao Norte,Espaillat,No declarado,Noviembre,2007,No declarado,Hombres,"Vivienda, domicilio o residencia","Sustancia tóxica, veneno","Conflicto familiar, violencia intrafamiliar y ...",No declarado
1,Región Cibao Norte,Espaillat,No declarado,Julio,2007,No declarado,Hombres,"Vivienda, domicilio o residencia","Sustancia tóxica, veneno",Otros problemas de salud,35 - 39
2,Región Cibao Norte,Espaillat,No declarado,Septiembre,2007,No declarado,Hombres,"Vivienda, domicilio o residencia",Arma de fuego,Otros,20 - 24
3,Región Cibao Norte,Espaillat,No declarado,Enero,2007,No declarado,Hombres,"Hotel, motel, pensión, cabaña",Arma de fuego,No declarado,30 - 34
4,Región Cibao Norte,Espaillat,No declarado,Febrero,2007,No declarado,Hombres,"Vía pública, parque","Sustancia tóxica, veneno",No declarado,20 - 24
...,...,...,...,...,...,...,...,...,...,...,...
10030,Región Ozama,Santo Domingo,Viernes,Enero,2021,18:01-20:00,Hombres,No declarado,Arma de fuego,"Depresión, problemas de salud mental",55 - 59
10031,Región Ozama,Santo Domingo,Lunes,Marzo,2018,08:01-10:00,Hombres,Instalaciones de negocio o empresa no comercia...,Arma de fuego,No declarado,40 - 44
10032,Región Ozama,Santo Domingo,Viernes,Diciembre,2018,08:01-10:00,Hombres,No declarado,"Ahorcamiento, asfixia respiratoria",No declarado,30 - 34
10033,No declarada,No declarada,Sábado,Noviembre,2018,18:01-20:00,Hombres,No declarado,"Sustancia tóxica, veneno",No declarado,40 - 44


In [14]:
df_lleno['provincia'].unique()

array(['Espaillat', 'Puerto Plata', 'Santiago', 'La Vega',
       'Sánchez Ramírez', 'Monseñor Nouel', 'Duarte',
       'María Trinidad. Sánchez', 'Hermanas Mirabal', 'Samaná', 'Dajabón',
       'Monte Cristi', 'Santiago Rodríguez', 'Valverde', 'Peravia',
       'San Cristóbal', 'San José de Ocoa', 'Baoruco', 'Barahona',
       'Independencia', 'Pedernales', 'Azua', 'Elías Piña', 'San Juan',
       'El Seibo', 'La Altagracia', 'La Romana', 'San Pedro de Macorís',
       'Monte Plata', 'Hato Mayor', 'Distrito Nacional', 'Santo Domingo',
       'No declarada'], dtype=object)

In [15]:
df_lleno['provincia'] = df_lleno['provincia'].str.replace('Baoruco', 'Bahoruco')
df_lleno['provincia'] = df_lleno['provincia'].str.replace('María Trinidad. Sánchez', 'María Trinidad Sánchez')

In [16]:
df_lleno.to_csv('../Datos/df_decodificado.csv')